In [13]:
import keras
import os
import json
import tensorflow as tf
import pandas as pd
import numpy as np

In [5]:
data_dir2='code/delft15_phase2/'

In [6]:
skill_test = pd.read_csv(data_dir2+"test_event_df_delft_15.csv").iloc[:, 1:].values
response_test = pd.read_csv(data_dir2+"test_response_df_delft_15.csv").iloc[:, 1:].values

In [7]:
skill_train = pd.read_csv(data_dir2+"train_event_df_delft_15.csv").iloc[:, 1:].values
response_train = pd.read_csv(data_dir2+"train_response_df_delft_15.csv").iloc[:, 1:].values

In [8]:
def find_accuracy(true_label, predicted_label):
    return np.mean(np.equal(true_label, predicted_label))

## Baseline - 1 (Global mode) - Majority response entire problem set

In [14]:
# Find the majority label in response_array_train 
# Create a new baseline prediction where all predicted values take the majority label 
# write an accuracy function
def baseline_1(response_array_train, response_array_test):
    total_elements = response_array_train.shape[0]* response_array_train.shape[1]
    number_of_ones = response_array_train.sum()
    number_of_zeros = total_elements - number_of_ones
    if number_of_ones > number_of_zeros:
        #majority_label = 1
        baseline_prediction = np.ones([response_array_test.shape[0], response_array_test.shape[1]])
    else:
        #majority_label = 0
        baseline_prediction = np.zeros([response_array_test.shape[0], response_array_test.shape[1]])
    return baseline_prediction

In [15]:
baseline_pred_1 = baseline_1(response_train, response_test)
print("Global Mode Baseline Accuracy: ",find_accuracy(response_test, baseline_pred_1))

Global Mode Baseline Accuracy:  0.84813


# Baseline - 2 (Mode question-wise) - Majority response per question

In [17]:
def prepare_qn_dict(skill_array_train, response_array_train):
    question_dict = {k:[0,0] for k in range(1,337)}
    #This is dictionary that has problem id as key and values 
    #that represent number of zeroes (incorrect) and number of ones (correct) respectively"
    for i in range(skill_array_train.shape[0]):
        for idx, prob_id in enumerate(skill_array_train[i]):
            if prob_id >= 1 and prob_id <= 336:
                if response_array_train[i][idx] == 0:
                    question_dict[prob_id][0] = question_dict[prob_id][0] + 1
                else: 
                    question_dict[prob_id][1] = question_dict[prob_id][1] + 1
    return question_dict

In [18]:
def baseline_2(question_dict, response_array_test, skill_array_test):
    num_student = response_array_test.shape[0]
    num_steps = response_array_test.shape[1]
    baseline_pred = np.zeros([num_student, num_steps])
    for i in range(skill_array_test.shape[0]):
        for idx, prob_id in enumerate(skill_array_test[i]): 
            if prob_id >= 1 and prob_id <= 336:
                majority_label = np.argmax(question_dict[prob_id])
                baseline_pred[i][idx]=majority_label
    return baseline_pred

In [19]:
question_dict = prepare_qn_dict(skill_train, response_train)
baseline_pred_2 = baseline_2(question_dict, response_test, skill_test)
print("Per Question Mode Baseline Accuracy: ",find_accuracy(response_test, baseline_pred_2))

Per Question Mode Baseline Accuracy:  0.89832
